<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/Python-Notebook-Banners/Code_challenge.png"  style="display: block; margin-left: auto; margin-right: auto;";/>
</div>

# Code challenge: Random forest regression
© ExploreAI Academy

In this code challenge, we'll test our knowledge of how to create an ensemble model known as a random forest. We will train this new model using the world population data.

⚠️ **Note that this code challenge is graded and will contribute to your overall marks for this module. Submit this notebook for grading. Note that the names of the functions are different in this notebook. Transfer the code in your notebook to this submission notebook**

### Instructions

- **Do not add or remove cells in this notebook. Do not edit or remove the `### START FUNCTION` or `### END FUNCTION` comments. Do not add any code outside of the functions you are required to edit. Doing any of this will lead to a mark of 0%!**

- Answer the questions according to the specifications provided.

- Use the given cell in each question to see if your function matches the expected outputs.

- Do not hard-code answers to the questions.

- The use of StackOverflow, Google, and other online tools is permitted. However, copying a fellow student's code is not permissible and is considered a breach of the Honour code. Doing this will result in a mark of 0%.

### Imports

In [1]:
import numpy as np
import pandas as pd
from numpy import array
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [2]:
population_df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/AnalyseProject/world_population.csv', index_col='Country Code')
meta_df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/AnalyseProject/metadata.csv', index_col='Country Code')

In [3]:
population_df.head()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country Code,,,,,,,,,,,,,,,,,,,,,
ABW,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,57715.0,58055.0,58386.0,58726.0,...,101353.0,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0,105264.0
AFG,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,10152331.0,10372630.0,10604346.0,10854428.0,...,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0,35530081.0
AGO,5643182.0,5753024.0,5866061.0,5980417.0,6093321.0,6203299.0,6309770.0,6414995.0,6523791.0,6642632.0,...,21759420.0,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463.0,29784193.0
ALB,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,1914573.0,1965598.0,2022272.0,2081695.0,...,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0
AND,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,19647.0,20758.0,21890.0,23058.0,...,83861.0,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281.0,76965.0


In [4]:
meta_df.head()

,Region,Income Group,Special Notes
Country Code,,,
ABW,Latin America & Caribbean,High income,Mining is included in agriculture\r\r\r\nElect...
AFG,South Asia,Low income,Fiscal year end: March 20; reporting period fo...
AGO,Sub-Saharan Africa,Lower middle income,NaN
ALB,Europe & Central Asia,Upper middle income,NaN
AND,Europe & Central Asia,High income,WB-3 code changed from ADO to AND to align wit...


### Question 1

The world population data spans from 1960 to 2017. We'd like to build a predictive model that can give us the best guess at what the world population in a given year was. However, we want to compute this estimate for only _countries within a given income group_.

First, however, we need to organise our data such that the sklearn's `RandomForestRegressor` class can train on our data. To do this, we will write a function that takes as input an income group and returns a 2-d numpy array that contains the year and the measured population.

_**Function Specifications:**_
* Should take a `str` argument, called `income_group_name` as input and return a numpy `array` type as output.
* Set the default argument of `income_group_name` to equal `'Low income'`.
* If the specified value of `income_group_name` does not exist, the function must raise a `ValueError`.
* The array should only have two columns containing the year and the population, in other words, it should have a shape `(?, 2)` where `?` is the length of the data.
* The values within the array should be of type `np.int64`.

_**Further Reading:**_

Data types are associated with memory allocation. As such, your choice of data type affects the precision of computations in your program. For example, the `np.int` data type in numpy can only store values between -2147483648 to 2147483647 and assigning values outside this range for variables of this data type may cause run-time errors. To avoid this, we can use data types with larger memory capacity e.g. `np.int64`.

https://docs.scipy.org/doc/numpy/user/basics.types.html

In [16]:
### START FUNCTION
def get_total_pop_by_income(income_group_name='Low income'):
    """
    This function returns a 2D numpy array containing the year and total population for countries in a specified income group.

    Parameters:
    income_group_name (str): The name of the income group. Default is 'Low income'.

    Returns:
    np.array: A 2D numpy array with the year and total population.

    Raises:
    ValueError: If the specified income group does not exist.
    """

    # Check if the income group exists in the metadata
    # If it doesn't, raise a ValueError
    if income_group_name not in meta_df['Income Group'].unique():
        raise ValueError(f"Income group '{income_group_name}' does not exist.")

    # Get the list of countries in the specified income group
    countries = meta_df[meta_df['Income Group'] == income_group_name].index

    # Filter the population data for these countries
    # Sum the population data across all countries for each year
    pop_data = population_df.loc[countries].sum()

    # Create a 2D numpy array with the year and total population
    # Ensure the data type is np.int64
    result = np.array([pop_data.index.astype(np.int64), pop_data.values.astype(np.int64)]).T

    return result
### END FUNCTION

In [18]:
data = get_total_pop_by_income('High income')

_**Expected Outputs:**_
```python
get_total_pop_by_income('High income')
```
> ```
array([[      1960,  769889923],
       [      1961,  781225329],
       [      1962,  791207437],
       [      1963,  801108277],
       ...
       [      2015, 1211252041],
       [      2016, 1218629612],
       [      2017, 1225514228]])
```




### Question 2

Now that we have have our data, we need to split this into a set of variables we will be training on, and the set of variables that we will make our predictions on.

Sklearn has a bunch of built-in functionality for creating training and testing sets. Our task is to implement a k-fold cross validation split of the data using sklearn's `KFold` [class](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) (which has already been imported into this notebook for your convenience).

Using this knowledge, write a function which uses sklearn's `KFold` [class](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) internally, and that will take as input a 2-d numpy array and an integer `K` corresponding to the number of splits. This function will then return a list of tuples of length `K`. Each tuple in this list should consist of a `train_indices` list and a `test_indices` list containing the training/testing data point indices for that particular $K^{th}$ split.

_**Function Specifications:**_
* Should take a 2-d numpy `array` and an integer `K` as input.
* Should use sklearn's `KFold` [class](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html).
* Should return a list of `K` `tuples` containing a list of training and testing indices corresponding to the data points that belong to a particular split. For example, given an array called `data` and an integer `K`, the function should return:
>```
data_indices = [(list_of_train_indices_for_split_1, list_of_test_indices_for_split_1)
                  (list_of_train_indices_for_split_2, list_of_test_indices_for_split_2)
                  (list_of_train_indices_for_split_3, list_of_test_indices_for_split_3)
                                                   ...
                                                   ...
                  (list_of_train_indices_for_split_K, list_of_test_indices_for_split_K)]
```

* The `shuffle` argument in the KFold object should be set to `False`.

**_Hint_**: To see an example of how to use the `KFold` object enter `help(KFold)` in a new notebook cell

In [19]:
### START FUNCTION
def sklearn_kfold_split(data, K):
    """
    This function splits the data into K folds for cross-validation.

    Parameters:
    data (np.array): The data to be split.
    K (int): The number of folds.

    Returns:
    list: A list of tuples. Each tuple contains the training and testing indices for one fold.
    """
    from sklearn.model_selection import KFold

    # Initialize the KFold object
    kf = KFold(n_splits=K, shuffle=False)

    # Get the training and testing indices for each fold
    folds = [(train_index, test_index) for train_index, test_index in kf.split(data)]

    return folds

### END FUNCTION

In [20]:
data = get_total_pop_by_income('High income');
sklearn_kfold_split(data,4)

[(array([15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
         32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
         49, 50, 51, 52, 53, 54, 55, 56, 57]),
  array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 30, 31,
         32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
         49, 50, 51, 52, 53, 54, 55, 56, 57]),
  array([15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 44, 45, 46, 47,
         48, 49, 50, 51, 52, 53, 54, 55, 56, 57]),
  array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38

_**Expected Outputs:**_
```python
data = get_total_pop_by_income('High income')
sklearn_kfold_split(data,4)
```
> ```
[(array([15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
         32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
         49, 50, 51, 52, 53, 54, 55, 56, 57]),
  array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 30, 31,
         32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
         49, 50, 51, 52, 53, 54, 55, 56, 57]),
  array([15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 44, 45, 46, 47,
         48, 49, 50, 51, 52, 53, 54, 55, 56, 57]),
  array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38, 39, 40, 41, 42, 43]),
  array([44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57]))]
 ```

### Question 3

Now that we have formatted our data, we can fit a model using sklearn's `RandomForestRegressor` class. We'll write a function that will take as input the data indices (consisting of the train and test indices for each split) that we created in the last question, train a different `RandomForestRegressor` on each split and return the model that obtains the best testing set performance across all K splits.

**Important Note:** Due to the random initialisation process used within sklearn's `RandomForestRegressor` class, you will need to fix the value of the `random_state` argument in order to get repeatable and predictable results.

_**Function Specifications:**_
* Should take a 2-d numpy array (i.e. the data) and `data_indices` (a list of `(train_indices,test_indices)` tuples) as input.
* For each `(train_indices,test_indices)` tuple in `data_indices` the function should:
    * Train a new `RandomForestRegressor` model on the portion of data indexed by `train_indices`
    * Evaluate the trained `RandomForestRegressor` model on the portion of data indexed by `test_indices` using the **mean squared error** (which has also been imported for your convenience).
* After training and evaluating the `RandomForestRegressor` models, the function should return the `RandomForestRegressor` model that obtained the highest testing set `mean_square_error` over its allocated data split across all trained models.
* The trained `RandomForestRegressor` models should be trained with `random_state` equal `42`, all other parameters should be left as default.

**_Hint_**: for each tuple in the `data_indices` list, you can obtain `X_train`,`X_test`, `y_train`, `y_test` as follows:  
>```
    X_train, y_train = data[train_indices,0],data[train_indices,1]
    X_test, y_test = data[test_indices,0],data[test_indices,1]
```



In [47]:
### START FUNCTION
def best_k_model(data, data_indices):
    """
    This function trains a RandomForestRegressor model on each split of the data and returns the model that obtains
    the best testing set performance across all K splits.

    Parameters:
    data (numpy.ndarray): The dataset used to train and test the model. It is a 2D array where the first column is the
                          feature (year) and the second column is the target (population).
    data_indices (list): A list of tuples. Each tuple contains two numpy arrays: the first array is the indices of the
                         training set and the second array is the indices of the testing set.

    Returns:
    RandomForestRegressor: The model that has the highest testing set mean_square_error over its allocated data split
                           across all trained models.
    """

    # Initialize the best mean square error to infinity
    best_mse = float('inf')

    # Initialize the best model to None
    best_model = None

    # Iterate over each tuple in the data_indices list
    for train_indices, test_indices in data_indices:

        # Extract the training set and testing set from the data
        X_train, y_train = data[train_indices, 0].reshape(-1, 1), data[train_indices, 1]
        X_test, y_test = data[test_indices, 0].reshape(-1, 1), data[test_indices, 1]

        # Initialize a RandomForestRegressor model with random_state equal to 42
        model = RandomForestRegressor(random_state=42)

        # Train the model on the training set
        model.fit(X_train, y_train)

        # Use the trained model to predict the target of the testing set
        y_pred = model.predict(X_test)

        # Calculate the mean square error of the model's prediction
        mse = mean_squared_error(y_test, y_pred)

        # If the current model's mean square error is less than the best mean square error
        if mse < best_mse:
            # Update the best mean square error
            best_mse = mse

            # Update the best model
            best_model = model

    # Return the best model
    return best_model
### END FUNCTION

In [49]:
data = get_total_pop_by_income('High income')
data_indices = sklearn_kfold_split(data,5)

best_model = best_k_model(data,data_indices)
best_model.predict([[1960]])

array([7.76089629e+08])

_**Expected Outputs:**_
```python
best_model.predict([[1960]]) == array([8.85170916e+08])
```

#  

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/EAI_Blue_Dark.png"  style="width:200px";/>
</div>